# Find Big Galaxies in DP0

Author(s): Alex Drlica-Wagner and DP0 Delegates Breakout Group

This notebook was the result of a breakout hack session during the DP0 delegate assembly on September 10, 2021. The goal was to try to find large galaxies in the DP0 coadd data. We chose to do a visual inspection of tracts in a single patch.

In [ ]:
# Generic python packages
import gc
import time
import numpy as np
import pylab as plt
import matplotlib.animation as animation
from IPython.display import display, clear_output, HTML

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [ ]:
# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom
import lsst.afw.coord as afwCoord
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Create a butler pointing to the DP0 data
repo = 's3://butler-us-central1-dp01'
collection = "2.2i/runs/DP0.1"
butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
ra, dec = 55.064, -29.783
radec = geom.SpherePoint(ra, dec, geom.degrees)

skymap = butler.get("skyMap")

tractInfo = skymap.findTract(radec)
tract = tractInfo.getId()

patchInfo = tractInfo.findPatch(radec)
patch = tractInfo.getSequentialPatchIndex(patchInfo)

print("tract: {tract}, patch: {patch}")

In [ ]:
def remove_figure(fig):
    """Remove a figure to reduce memory footprint. """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()      # clear the figure
    plt.close(fig) # close the figure
    gc.collect()   # call the garbage collector

We could either loop through the 49 patches and get the coadd from the butler or we can run one query and then loop through it. The first option is easier to understand, and is the approach we take; however, the second is more efficient.

In [ ]:
# First, let's make some gray-scale images
fig,ax = plt.subplots()

# Run the full image set?
test=True

for i,patch in enumerate(range(49)):
    data_id = {'tract': tract, 'patch': patch, 'band':'i'}
    coadd = butler.get('deepCoadd',dataId=data_id)
    
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(coadd.image)
    plt.gca().axis('off')
    plt.title(str(data_id),fontsize=12)

    # To draw into the same frame...
    clear_output(wait=True)
    display(fig)
    remove_figure(fig)

    if test and i > 1: break

In [ ]:
# The more efficient option is to query upfront
dataIds = {'tract': tract, 'band': 'i'}
coadd_refs = butler.registry.queryDatasets('deepCoadd',dataId=dataIds)
coadd_refs = sorted(coadd_refs)

# Testing
test = True

fig,ax = plt.subplots()
for i,ref in enumerate(coadd_refs):
    coadd = butler.getDirect(ref)
    
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(coadd.image)
    plt.gca().axis('off')
    plt.title(str(ref.dataId),fontsize=12)

    # To draw into the same frame...
    clear_output(wait=True)
    display(fig)
    remove_figure(fig)
    
    if test and i > 1: break  

Next we try to make some color images. This requires us to query three bands and create a `MultibandExposure` object. We copy over the `createRGB` function from the [03_Image_Display_and_Manipulation.ipynb](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/03_Image_Display_and_Manipulation.ipynb) tutorial notebook. We also borrow a trick from the Stack Club [Footprints.ipynb](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/SourceDetection/Footprints.ipynb) notebook to animate the resulting RGB images.

In [ ]:
from astropy.visualization import make_lupton_rgb
from lsst.afw.image import MultibandExposure

def createRGB(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].image.array  # numpy array for the r channel
        g_im = image[bgr[1]].image.array  # numpy array for the g channel
        b_im = image[bgr[0]].image.array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].image.array * scale[0]
        g_im = image[bgr[1]].image.array * scale[1]
        b_im = image[bgr[0]].image.array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
bands = ['g','r','i']

# Borrowing a trick from the Stack Club Footprints.ipynb...
# Images is a list of images that can be animated
images = []

test = True

fig,ax = plt.subplots()
for i,patch in enumerate(range(49)):
    data_id = {'tract': 4638, 'patch': patch}
    print(data_id)
    coadds = []
    for band in bands:
        data_id['band'] = band
        coadds.append(butler.get('deepCoadd',dataId=data_id))

    coadds = MultibandExposure.fromExposures(bands, coadds)
    rgb = createRGB(coadds)

    # Store images in a list to animate later
    im = ax.imshow(rgb, origin='lower', animated=True, visible=False)
    images.append([im])

    if test and i > 2: break
        
# close a spurious extra image that is created
plt.close()

# makes an animation of the parent, then child footprints
HTML(animation.ArtistAnimation(fig, images, interval=1000, blit=True,
                                repeat_delay=1000).to_jshtml())
# takes some time to get going... 